In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import math


# Print the installed solver to see if GUROBI is installed correctly
print("Current installed solvers: ", cp.installed_solvers())

2024-08-23 09:42:10.352589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Current installed solvers:  ['CLARABEL', 'ECOS', 'ECOS_BB', 'GUROBI', 'OSQP', 'SCIPY', 'SCS']


In [2]:
# Load the data
load_data = pd.read_csv('data/CAISO_Load_2022.csv', usecols=['net_load']).values.flatten()
thermal_gen_data = pd.read_csv('data/thermal_gen_offer.csv', usecols=['capacity_MW', 'energy_price'])

In [4]:
# Parameters
num_days = 365
num_steps = 288
num_intervals = 12
P = 10_000  # Total storage capacity in MW
E = 40_000  # Total storage capacity in MWh
eta = 0.9  # Storage one-way efficiency
C_s = 0.0  # Storage marginal discharge cost

# Generator data 
P_g = thermal_gen_data['capacity_MW'].values # Thermal generator capacities (MW)
C_g = thermal_gen_data['energy_price'].values # Thermal generator offer ($/MW)

# Storage segment data
num_seg = 5

# Reshape load data for num_days of num_steps
load = load_data[0:num_days*num_steps].reshape((num_days, num_steps)) # Net load profile

# Number of generators
num_gen = len(P_g)

In [6]:
# Function to solve the optimization problem for a single period
def solve_period_optimization(L, P_g, C_g, P, E, eta, C_s, E0, num_intervals, num_seg, soc_bids):
    # Variables
    p = cp.Variable(num_gen)  # Power generation by each generator
    c = cp.Variable(num_seg)             # Power charge by storage
    d = cp.Variable(num_seg)             # Power discharge by storage
    e = cp.Variable(num_seg)             # State of Charge (SoC) of storage

    # Constraints
    constraints = []

    # Generator capacity constraints
    for g in range(num_gen):
        constraints += [p[g] >= 0,
                        p[g] <= P_g[g]]

    # Storage constraints
    for s in range(num_seg):
        constraints += [c[s] >= 0,
                        c[s] <= P / num_seg,
                        d[s] >= 0,
                        d[s]<= P / num_seg]

    # SoC constraints
    for s in range(num_seg):
        constraints += [e[s] == E0[s] + eta * c[s] - d[s] / eta]
        constraints += [e[s] <= E / num_seg]
        constraints += [e[s] >= 0]
        
    # Balance constraints and capture dual variables
    balance_constraint = cp.sum(p) + cp.sum(d) == L + cp.sum(c)
    constraints += [balance_constraint]

    # Objective: Minimize system cost
    cost = (cp.sum(cp.multiply(p, C_g.flatten())) +
            cp.sum(cp.multiply(E0-e, soc_bids)) +
            cp.sum(C_s * d)) / num_intervals
    objective = cp.Minimize(cost)

    # Problem definition
    problem = cp.Problem(objective, constraints)

    # Solve the problem using Gurobi if available, otherwise use a different solver
    start_time = time.time()
    try:
        problem.solve(solver=cp.GUROBI, verbose=False, reoptimize=True)
        dual_price = balance_constraint.dual_value * -num_intervals  # Extract dual values
    except cp.SolverError:
        print("Gurobi not available. Falling back to a different solver.")
        problem.solve(solver=cp.ECOS, verbose=False)  # You can use ECOS, SCS, or another solve
        dual_price = balance_constraint.dual_value  * -num_intervals  # Extract dual values
    end_time = time.time()

    return problem.value, p.value, c.value, d.value, e.value, dual_price, end_time - start_time

In [11]:
# Solve for each time step
results = []

all_daily_costs = []
all_gen_output = []
all_storage_ops = []
all_clearing_prices= []

total_start_time = time.time()  # Start time for the entire solving loop

model = tf.keras.models.load_model('models/model0.h5')


for day in range(num_days):
    # solve daily for first day (288 periods) to generate initial price signals
    L = load[day, :]
    if day == 0:
        day = 0
        E0 = E / 2
        daily_cost, daily_gen, daily_charge, daily_discharge, daily_soc, dual_prices, run_time = solve_daily_optimization(L, P_g, C_g, P, E, eta, C_s, E0, num_intervals)
        results.append((daily_cost, daily_gen, daily_charge, daily_discharge, daily_soc))
        print(f"Day {day + 1}: Cost = {daily_cost}, Run Time = {run_time} seconds")
    
        # Collect results for each day
        all_daily_costs.append({
            'day': day + 1,
            'cost': daily_cost,
            'run_time': run_time
        })
    
        all_gen_output.append(pd.DataFrame(daily_gen, columns=[f'gen_{i+1}' for i in range(num_gen)]))
        all_storage_ops.append(pd.DataFrame({
            'charge': daily_charge,
            'discharge': daily_discharge,
            'soc': daily_soc
        }))
        all_clearing_prices.append(pd.DataFrame({'clearing_price': dual_prices}))
    else:
        for ts in range(num_steps):
            if (day == 1) & (ts == 0):
                E0 = [0.2 * E, 0.2 * E, 0.1 * E, 0.0, 0.0]
            else:
                continue
            v = model.predict(dual_prices).T
            soc_bids = np.mean(v.reshape(num_seg, int(v.shape[0]/num_seg)), axis=1, keepdims=True).flatten()
            daily_cost, daily_gen, daily_charge, daily_discharge, daily_soc, dual_price, run_time = solve_period_optimization(L, P_g, C_g, P, E, eta, C_s, E0, num_intervals, num_seg, soc_bids)


total_end_time = time.time()  # End time for the entire solving loop
total_run_time = total_end_time - total_start_time

print(f"Total Run Time for all days: {total_run_time} seconds")


Day 1: Cost = 15792525.42701489, Run Time = 0.9929580688476562 seconds


ValueError: in user code:

    File "/Users/ningkunzheng/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/ningkunzheng/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ningkunzheng/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/ningkunzheng/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/ningkunzheng/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/ningkunzheng/opt

In [59]:
model = tf.keras.models.load_model('models/model0')
model